In [8]:
pip install beautifulsoup4

     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import requests

<h4>Scrape the wikipedia page to get the contnets of the page</h4>

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page_postalCanada=requests.get(url)
page_postalCanada

<Response [200]>

<h4> Using BeautifulSoup first find the table and then fetch all contents in the tag tr to rows</h4>

In [3]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(page_postalCanada.content,'html.parser')
table_postalCanada=soup.find("table")
rows=table_postalCanada.find_all("tr")

<h4>Transform the data into Pandas Dataframe</h4>

In [4]:
postal_codes=pd.DataFrame([pt.get_text() for pt in rows])
print(postal_codes)

                                             0
0         \nPostcode\nBorough\nNeighbourhood\n
1          \nM1A\nNot assigned\nNot assigned\n
2          \nM2A\nNot assigned\nNot assigned\n
3               \nM3A\nNorth York\nParkwoods\n
4        \nM4A\nNorth York\nVictoria Village\n
..                                         ...
284              \nM8Z\nEtobicoke\nMimico NW\n
285     \nM8Z\nEtobicoke\nThe Queensway West\n
286  \nM8Z\nEtobicoke\nRoyal York South West\n
287         \nM8Z\nEtobicoke\nSouth of Bloor\n
288        \nM9Z\nNot assigned\nNot assigned\n

[289 rows x 1 columns]


<h4>Format the data to the desired table form</h4>

In [5]:
df=postal_codes[0].str.split("\n",n=4,expand=True)
df.drop(columns=4,inplace=True)
new_header=df.iloc[0]
df=df[1:]
df.columns=new_header
df.shape

(288, 4)

In [6]:
df.head()

,,Postcode,Borough,Neighbourhood
1,,M1A,Not assigned,Not assigned
2,,M2A,Not assigned,Not assigned
3,,M3A,North York,Parkwoods
4,,M4A,North York,Victoria Village
5,,M5A,Downtown Toronto,Harbourfront


<h4>Drop the rows where Borough is 'Not assigned'</h4>

In [7]:
df=df[df.Borough != "Not assigned"]
df.shape

(211, 4)

In [8]:
df.head()

,,Postcode,Borough,Neighbourhood
3,,M3A,North York,Parkwoods
4,,M4A,North York,Victoria Village
5,,M5A,Downtown Toronto,Harbourfront
6,,M5A,Downtown Toronto,Regent Park
7,,M6A,North York,Lawrence Heights


<h4>Combine the rows where more than one neighborhood exist in one postal code area.</h4>

In [9]:
df_final=df.groupby("Postcode").agg(lambda x:','.join(set(x)))

In [10]:
df_final.reset_index()
df_final.shape

(103, 3)

In [11]:
df_final.head()

,,Borough,Neighbourhood
Postcode,,,
M1B,,Scarborough,"Malvern,Rouge"
M1C,,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,,Scarborough,"Morningside,Guildwood,West Hill"
M1G,,Scarborough,Woburn
M1H,,Scarborough,Cedarbrae


<h4>Assign the value of Borough to Neighborhood incase Borough has value but a "Not assigned" Neighbourhood</h4>

In [12]:
df_final.Neighbourhood[df_final.Neighbourhood == "Not assigned"]=df_final.Borough

In [13]:
df_final.shape

(103, 3)

<h2>Assignment Part 2 Adding Latitude and Longitude to the Postal Codes</h2>

<h3>Reading Latitude and Longitude values from the Excel file</h3>

In [14]:
import pandas as pd

In [15]:
df_lat_long=pd.read_csv("http://cocl.us/Geospatial_data")

In [16]:
df_lat_long.shape

(103, 3)

In [17]:
df_merge=pd.merge(df_lat_long,df_final,left_on='Postal Code',right_on="Postcode")

In [18]:
df_merge

,Postal Code,Latitude,Longitude,,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,,Scarborough,"Malvern,Rouge"
1,M1C,43.784535,-79.160497,,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,43.763573,-79.188711,,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,43.770992,-79.216917,,Scarborough,Woburn
4,M1H,43.773136,-79.239476,,Scarborough,Cedarbrae
...,...,...,...,...,...,...
98,M9N,43.706876,-79.518188,,York,Weston
99,M9P,43.696319,-79.532242,,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,,Etobicoke,"Richview Gardens,Kingsview Village,St. Phillip..."
101,M9V,43.739416,-79.588437,,Etobicoke,"Mount Olive,South Steeles,Thistletown,Silverst..."


In [19]:
df_merge=df_merge.reindex(columns=['Postal Code','Borough','Neighbourhood','Latitude','Longitude'])

In [20]:
df_merge

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Richview Gardens,Kingsview Village,St. Phillip...",43.688905,-79.554724
101,M9V,Etobicoke,"Mount Olive,South Steeles,Thistletown,Silverst...",43.739416,-79.588437
